In [1]:
import pandas as pd
import numpy as np

In [2]:
vendor_mapping = pd.read_excel(r'C:\Medline\2. CPM\data\vendor_mapping\Vendor _mapping 2024_v1.xlsx')
vendor_mapping['Vendor Number'] = vendor_mapping['Vendor Number'].map(str)
vendor_mapping_dict = dict(zip(vendor_mapping['Vendor Number'],vendor_mapping['Cleaned Vendor Name']))

In [3]:
vendor_mapping_dict = dict(zip(vendor_mapping['Vendor Number'],vendor_mapping['Cleaned Vendor Name']))
vendor_nation_dict = dict(zip(vendor_mapping['Cleaned Vendor Name'],vendor_mapping['Nation']))
vendor_manager_dict = dict(zip(vendor_mapping['Cleaned Vendor Name'],vendor_mapping['Regional Manager']))

In [4]:
blitz_vendor_info = pd.read_excel(r'C:\Medline\Vendor info\2024_BlitzVendor.xlsx')
blitz_ProjectLeader_dict = dict(zip(blitz_vendor_info['Vendor Name'],blitz_vendor_info['Project Leader']))
blitz_Category_dict = dict(zip(blitz_vendor_info['Vendor Name'],blitz_vendor_info['Category']))

In [5]:
df_complaints =pd.read_excel(r'C:\Medline\2. CPM\2. US Complaints\US Complaint Data 201901-202312.xlsx',sheet_name=1)

C:\Users\ypeng\AppData\Local\Temp\ipykernel_13036\2473699971.py:1: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  df_complaints =pd.read_excel(r'C:\Medline\2. CPM\2. US Complaints\US Complaint Data 201901-202312.xlsx',sheet_name=1)


In [6]:
df_complaints_gy = (
    df_complaints
    .query("`If Manufacturing Complaint` == 'Y' and Year.isin([2023])")
    .assign(Quarter =lambda d: d['Notification Created Date'].dt.quarter,
            Division = lambda d: d['Division'].map(str))
    .groupby(['Vendor Name','Year','Quarter','Month'],as_index=False).size()
)
df_complaints_gy.rename(columns={'size':'Complaints','Vendor Name':'VendorName'},inplace=True)

In [7]:
df_inspection_his = pd.read_excel(r'C:\Medline\3. QC trend report\Asia QC Trend Report 202001_202312.xlsx',sheet_name=0)

In [8]:
df_inspection_us_his =  df_inspection_his.loc[(df_inspection_his['Branch'] == 'US' ) & (df_inspection_his['Year'] == 2023),['Vendors','Year','Quarter','Month','#: Items inspected','#: Items reworked']]

In [9]:
# df_inspection = pd.concat([df_inspection_us_his,df_inspection_us_current],ignore_index=True)
df_inspection_us_groupby = df_inspection_us_his.groupby(['Vendors','Year','Quarter','Month'],as_index=False).sum()

In [10]:
df_inspection_us_groupby.rename(columns={'Vendors':'VendorName'},inplace=True)

In [11]:
df_us_buom_2023 = pd.read_excel(r'C:\Medline\2. CPM\4. CPM by BUOM\BUOM Database\history\US_ShippingData_2023.xlsx',usecols='D,E,AB,AP,BI')
df_us_buom_2024 = pd.read_excel(r'C:\Medline\2. CPM\4. CPM by BUOM\BUOM Database\2024\US\US BUOM 202401.xlsx',usecols='D,E,P,X,Y')
df_us_buom = pd.concat([df_us_buom_2024,df_us_buom_2023],ignore_index=True)
df_us_buom.rename(columns={'StrippedVendorNumber':'VendorNumber'},inplace=True)
df_us_buom['VendorNumber'] = df_us_buom['VendorNumber'].map(str)

In [12]:
vendor_mapping_inspection = vendor_mapping.loc[(~vendor_mapping['Regional Manager'].isin(['Exemption','US vendor']))&(vendor_mapping['Regional Manager'].notnull()),'Vendor Number'].to_list()
df_us_buom = df_us_buom.loc[df_us_buom['VendorNumber'].isin(vendor_mapping_inspection)]
df_us_buom['VendorName'] = df_us_buom['VendorNumber'].apply(lambda x : vendor_mapping_dict.get(x,np.nan))

In [13]:
df_us_buom['Year'] = df_us_buom['ShipmentDt'].dt.year
df_us_buom['Quarter'] = df_us_buom['ShipmentDt'].dt.quarter
df_us_buom['Month'] = df_us_buom['ShipmentDt'].dt.month

In [14]:
df_us_buom_gy = df_BUOM_gy = df_us_buom.groupby(['VendorName','Year','Quarter','Month'],as_index=False)['BUOMShipmentQty'].sum()

In [15]:
df_us_buom_gy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4506 entries, 0 to 4505
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   VendorName       4506 non-null   object
 1   Year             4506 non-null   int64 
 2   Quarter          4506 non-null   int64 
 3   Month            4506 non-null   int64 
 4   BUOMShipmentQty  4506 non-null   int64 
dtypes: int64(4), object(1)
memory usage: 176.1+ KB


In [16]:
df_complaints_gy

,VendorName,Year,Quarter,Month,Complaints
0,A AND J HOLDING CO LTD,2023,1,1,10
1,A AND J HOLDING CO LTD,2023,1,2,3
2,A AND J HOLDING CO LTD,2023,1,3,5
3,A AND J HOLDING CO LTD,2023,2,4,3
4,A AND J HOLDING CO LTD,2023,2,5,7
...,...,...,...,...,...
819,ZHONGSHAN BLISS MEDICAL,2023,3,8,13
820,ZHONGSHAN BLISS MEDICAL,2023,3,9,3
821,ZHONGSHAN BLISS MEDICAL,2023,4,10,6
822,ZHONGSHAN BLISS MEDICAL,2023,4,11,6


In [17]:
raw_data = df_complaints_gy.merge(df_us_buom_gy,how = 'outer',on=['VendorName','Year','Quarter','Month']).merge(df_inspection_us_groupby,how = 'outer',on=['VendorName','Year','Quarter','Month']).fillna(0)

In [18]:
raw_data['Nation'] = raw_data['VendorName'].apply(lambda x : vendor_nation_dict.get(x,np.nan))
raw_data['Regional Manager'] = raw_data['VendorName'].apply(lambda x : vendor_manager_dict.get(x,np.nan))
raw_data['Project Leader'] = raw_data['VendorName'].apply(lambda x : blitz_ProjectLeader_dict.get(x,np.nan))
raw_data['Category'] = raw_data['VendorName'].apply(lambda x : blitz_Category_dict.get(x,np.nan))
raw_data['CPM'] = None

In [19]:
raw_data.head()

,VendorName,Year,Quarter,Month,Complaints,BUOMShipmentQty,#: Items inspected,#: Items reworked,Nation,Regional Manager,Project Leader,Category,CPM
0,A AND J HOLDING CO LTD,2023.0,1.0,1.0,10.0,1826.0,13.0,0.0,CN,Richard Li(DME),Gaming Chen,Top Complaints,None
1,A AND J HOLDING CO LTD,2023.0,1.0,2.0,3.0,1439.0,32.0,0.0,CN,Richard Li(DME),Gaming Chen,Top Complaints,None
2,A AND J HOLDING CO LTD,2023.0,1.0,3.0,5.0,2279.0,17.0,1.0,CN,Richard Li(DME),Gaming Chen,Top Complaints,None
3,A AND J HOLDING CO LTD,2023.0,2.0,4.0,3.0,1237.0,18.0,1.0,CN,Richard Li(DME),Gaming Chen,Top Complaints,None
4,A AND J HOLDING CO LTD,2023.0,2.0,5.0,7.0,1236.0,18.0,1.0,CN,Richard Li(DME),Gaming Chen,Top Complaints,None


In [20]:
raw_data = raw_data[['VendorName',
                     'Nation',
                     'Project Leader',
                     'Regional Manager',
                     'Category',
                     'Year',
                     'Month',
                     'Quarter',
                     'Complaints',
                     'BUOMShipmentQty',
                     'CPM',
                     '#: Items inspected',
                     '#: Items reworked'
                     ]]

In [21]:
raw_data.to_excel('us_raw_data.xlsx')